# ASC Accelerometer analysis via Summary Statistics 

FDeveloping moving average acceleration and moving Standard Deviation for each data set.

Can run through a whole folder, or through random selections from the folder

## This is attempting a slicker method using matrix multiplication.  With that can then attempt GPU parallelization

In [11]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp
import pandas as pd

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    #location = 'E:\\Documents\\Dan\\Code\\Prospectus\\Document\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    location = 'E:\\Documents\\Dan\\Phd\\Play\\'

#Standard cycle for collors and line styles
#default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
#plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

In [12]:
#Extra Headers:
import os as os
import statistics as st
import os as os
import pandas as pd
import random
import multiprocessing
from joblib import Parallel, delayed
import time


In [13]:
#Columns names for a file with all 6 dimmensions
Header = np.array(['T', 'X','Y','Z','R','Theta','Phi'])

In [26]:
#opens all files in a folder and returns the 3 axis acceleration in one array
# made to run in parallel

def getAcceleration(FileName):
    try:
        DataSet = np.genfromtxt(open(FileName,'r'), delimiter=',',skip_header=0)
        return [[FileName[:7]+FileName[-5:-4],'x',DataSet[:,3]],[FileName[:7]+FileName[-5:-4],'y',DataSet[:,4]],[FileName[:7]+FileName[-5:-4],'z',DataSet[:,5]]]
    except:
        return [False,FileName,False]


In [15]:
#first attempt at making stats by brute force, very slow
def makeStats(DataArray):
    try:
        Arange = 50
        length = np.shape(DataArray[2])[0]
        StdDev = np.zeros(length)
        for j in range(length-Arange):
            k = (length-1)-j
            DataArray[2][k] = np.average(DataArray[2][k-Arange:k])
            StdDev[k]=st.stdev(DataArray[2][k-Arange:k])
        return [DataArray[0],DataArray[1],max(DataArray[2]),max(StdDev)]
    except:
        return ['','','','']

In [16]:
# Uses matrixes and numpy to calculate the standard deviations of each row simultaneously.
# designed to run each time step independently
# can be run in parallel

def MatStdDev(MoveMatrix, AvgVector):
    width = np.shape(MoveMatrix)[1]

    AvgMatrix = np.concatenate((AvgVector,AvgVector),axis=1)
    for i in range(width-2):
        AvgMatrix = np.concatenate((AvgMatrix,AvgVector),axis=1)
    AvgVector = []

    Deltas = MoveMatrix - AvgMatrix
    AvgMatrix = []
    MoveMatrix = []

    SquareDifference = np.power(Deltas,2)
    Deltas = []

    one = np.ones((width))

    SumSquares = SquareDifference.dot(one)
    SquareDifference = []

    StdDev = np.power(SumSquares, 0.5) / width
    SumSquares = []

    return StdDev

In [17]:
#Inputs:
folder1 = 'C:\\Users\\Hendrickson\\Desktop\\Phone Acceleration\\'
folder2 = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'

folder1 = "S:\\"
folder2 = "S:\\"

folder1 = '/sciclone/scr10/dchendrickson01/Recordings2/SubSet/'
folder2 = '/sciclone/scr10/dchendrickson01/Recordings2/SubSet/'
files = os.listdir(folder2)

Groups = 0
GroupSize = 50

RollingSize = 5

In [18]:
#Pre Calculations

num_cores = multiprocessing.cpu_count() - 2
if np.shape(files)[0] < GroupSize:
    GroupSize = np.shape(files)[0]

if Groups !=0:
    files = random.sample(files,GroupSize*Groups-1)

if np.size(files) % GroupSize ==0:
    loops = int(np.size(files)/GroupSize)
else:
    loops = int(float(np.size(files))/float(GroupSize))+1
start = time.time()

inverseRollingSize = 1 / RollingSize

In [ ]:
if __name__ == "__main__":
AllEvents=[]
Fails = []
#    for i in range(loops):
i=0

In [ ]:
AllAccels = Parallel(n_jobs=num_cores)(delayed(getAcceleration)(folder2+file) for file in files[i*GroupSize:((i+1)*GroupSize)])
MetaData = []
DataOnlyMatrix = []

In [27]:
for j in range(np.shape(AllAccels)[0]):
    if AllAccels[j][0] == False :
        if AllAccels[j][1][4:9] =='Point':
            print(j,AllAccels[j][1])
    else: 
        for k in range(3):
            MetaData.append([AllAccels[j][k][0], AllAccels[j][k][1]])
            if np.shape(np.matrix(AllAccels[j][k][2]))[1] < 60000:
                temp = np.pad(np.matrix(AllAccels[j][k][2]),(60000-np.shape(np.matrix(AllAccels[j][k][2]))[1],0))[0]
                temp=np.matrix(temp)
                if np.size(DataOnlyMatrix) == 0:
                    DataOnlyMatrix =np.matrix(temp)
                    temp=[]
                else:
                    DataOnlyMatrix = np.concatenate((DataOnlyMatrix,temp),axis=0)
                    temp=[]
            else:
                if np.size(DataOnlyMatrix) == 0:
                    DataOnlyMatrix =np.matrix(AllAccels[j][k][2])
                else:
                    DataOnlyMatrix = np.concatenate((DataOnlyMatrix,np.matrix(AllAccels[j][k][2])),axis=0)

DataOnlyMatrix = np.matrix(DataOnlyMatrix)
        

/usr/local/anaconda3-2021.11/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 11745763 and the array at index 1 has size 26036970

In [24]:
length = np.shape(DataOnlyMatrix)[1]
halfLength = int(length / 2)

print(length, halfLength, np.shape(AllAccels))

0 0 (8, 3)


In [25]:
AllAccels[0][0]

False

In [ ]:
Weights1 = np.zeros((halfLength,halfLength))
Weights2 = np.zeros((halfLength,halfLength+RollingSize-1))

for j in range(halfLength):
    r = j+1

    if r < RollingSize:
       ir = 1/ r
       for k in range(r):
            Weights1[k,j] = ir
    else:
        for k in range(RollingSize):
            Weights1[r-RollingSize+k, j] = inverseRollingSize
    for k in range(RollingSize):
        Weights2[j,j+k] = inverseRollingSize

FirstBit = np.dot(DataOnlyMatrix[:,:halfLength], Weights1)
Weights1 = []

In [ ]:
if length % 2 == 1: halfLength += 1
Weights2=Weights2[:,:halfLength]
SecondBit = np.dot(DataOnlyMatrix[:,halfLength:], Weights2)

Weights2 = [] 

Weights3 = np.zeros((halfLength,halfLength))
for j in range(RollingSize-1):
    for k in range(RollingSize-j-1):
        Weights3[halfLength-k-1,j] = inverseRollingSize

SecondBit = SecondBit + np.dot(DataOnlyMatrix[:,halfLength:], Weights3)
Weights3 = []

In [20]:
Averages = np.concatenate((FirstBit, SecondBit), axis=1)
FirstBit = []
SecondBit = []

# Calculating the Standard Devation by square
AllStdDevs = Parallel(n_jobs=num_cores)(delayed(MatStdDev)(DataOnlyMatrix[:,i:RollingSize+i],Averages[:,RollingSize+i] ) for i in range(length-RollingSize))
AllStdDevs=np.squeeze(AllStdDevs)
AllStdDevs = np.concatenate( (np.zeros((RollingSize,np.shape(AllStdDevs)[1])), AllStdDevs),axis=0)

one = np.ones((length))
AvgVector = Averages.dot(one) / np.shape(Averages)[0]
#Averages = []

AllStdDevs = np.transpose(AllStdDevs)
StdDevVector = AllStdDevs.dot(one) / np.shape(AllStdDevs)[0]
#AllStdDevs = []

if np.size(AllEvents) == 0:
    temp = np.concatenate((MetaData, AvgVector.transpose()), axis=1)
    AllEvents = np.concatenate((temp, np.matrix(StdDevVector).transpose()),axis = 1)
else:
    temp = np.concatenate((MetaData, AvgVector.transpose()), axis=1)
    temp = np.concatenate((temp, np.matrix(StdDevVector).transpose()),axis = 1)
    AllEvents = np.concatenate((AllEvents,temp),axis=0)
    temp=[]
print('Loop ' + str(i+1) + ' of ' + str(loops) + '.  Total time is ' + str((time.time() - start) / 60) + ' min')

0 0


IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
df = pd.DataFrame(data=AllEvents)
df.to_csv(folder1 + 'StatisticsReport2.csv', sep=',', index = False, header=False,quotechar='"')
